In [1]:
import psycopg2
import numpy as np
import pandas as pd

In [2]:
# Establishing connection
# Old database
# conn = psycopg2.connect(
#     database = "siyavula_intern_db2",
#     user="master_db2",
#     password="master1234",
#     host="siyavula-postgre-db2.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com",
#     port="5432")

conn = psycopg2.connect(
    database = "siyavula-intern-final",
    user="master_db2",
    password="master1234",
    host="siyavula-postgre-db2.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com",
    port="5432")

cursor=conn.cursor()

### Queries to data for baseline survey analysis

#### Assignments with questions from the baseline examined as part of their content
Dataset contains are previous assignments that have ever had atleast even on question from the baseline assessment examined as part of their questions. This includes also the baseline assignment (assignment_id = 22620)
It also gives a summary of the total number questions from the baseline assignment that were included in the assignment in the column ```n_of_bs_quizs```

In [3]:
# Let's query the assignments of that have had any of the
# baseline questions examined by as part of them
# The baseline assessment is 22620 assignment
baseline_assignments_query = """
    with assignments_with_baseline_quizs as (
        -- assignments with content baseline questions as part of their content
        select 
            assignment_id, count(distinct template_id) as n_of_bs_quizs
        from assignment_content ac1
        where exists (
            select template_id, section_id, assignment_id
            from assignment_content ac2
            where ac2.template_id = ac1.template_id
                and ac2.section_id = ac1.section_id
                and ac2.assignment_id = 22620)
        group by assignment_id)
    select 
        abq.assignment_id, ass.title, ass.owner_uuid,
        ass.created_at, ass.sent_at, abq.n_of_bs_quizs
    from assignments_with_baseline_quizs abq
    left join assignments ass on ass.id = abq.assignment_id
    order by abq.assignment_id;
"""
baseline_assignments = pd.read_sql(baseline_assignments_query, conn)
baseline_assignments.head()

,assignment_id,title,owner_uuid,created_at,sent_at,n_of_bs_quizs
0,1,Maths Assignment,29d167ae-bb6b-48a3-9445-0219153a5f7d,2017-10-27 07:39:02.214647,NaT,4
1,2,Maths Assignment,255d3b5e-1eaf-4983-b7d9-5b8fc2da0586,2017-10-27 07:47:23.916375,NaT,1
2,11,Maths Assignment,33a5bde7-9920-4ecb-a259-452c6907650b,2017-10-30 14:01:36.720957,NaT,1
3,14,Maths Assignment,255d3b5e-1eaf-4983-b7d9-5b8fc2da0586,2017-11-01 08:23:08.637389,NaT,1
4,15,Maths Assignment,255d3b5e-1eaf-4983-b7d9-5b8fc2da0586,2017-11-01 08:30:53.323463,NaT,2


In [4]:
# Export the Baseline assessment content (questions and related content)
baseline_assignments.to_csv('raw_data/assignments.csv', index=False)

#### Baseline assignment questions data
Dataset has more information on the baseline assessment content (questions) i.e. subject, book, chapter, section and topic examined by each question in the baseline.

In [5]:
# Let's query the baseline assignment templates (questions) 
baseline_assignment_questions_query = """
    select
        ac.id as question_id, se.chapter_id, ac.section_id, ac.template_id,
        su.full_name as subject, bk.title as book_title, ch.title as chapter_title, 
        se.title as section_title, tp.title as topic_title, tp.total_marks, 
        ch.difficulty as chapter_difficulty, se.difficulty as section_difficulty
    from assignment_content ac
    left join templates tp on ac.template_id = tp.id
    left join sections se on ac.section_id = se.id
    left join chapters ch on se.chapter_id = ch.id
    left join books bk on ch.book_id =  bk.id
    left join subjects su on su.id = bk.subject_id
    where assignment_id = 22620
    order by question_id;
"""
baseline_assignment_questions = pd.read_sql(baseline_assignment_questions_query, conn)
baseline_assignment_questions.head()

,question_id,chapter_id,section_id,template_id,subject,book_title,chapter_title,section_title,topic_title,total_marks,chapter_difficulty,section_difficulty
0,400878,29,204,1762,Mathematics,"Everything Maths, Grade 8",Whole numbers,Properties of whole numbers,Inequalities: comparing two numbers,1,4.0,3.5
1,400879,29,230,1766,Mathematics,"Everything Maths, Grade 8",Whole numbers,Calculations using whole numbers,"Sum, difference, product, and quotient",2,4.0,3.5
2,400880,29,230,4455,Mathematics,"Everything Maths, Grade 8",Whole numbers,Calculations using whole numbers,Order of operations: <latex>a + b \times c</la...,2,4.0,3.5
3,400881,29,230,4466,Mathematics,"Everything Maths, Grade 8",Whole numbers,Calculations using whole numbers,Word sums with whole numbers,2,4.0,3.5
4,400882,29,254,1731,Mathematics,"Everything Maths, Grade 8",Whole numbers,Prime factorisation,Factors and multiples,2,4.0,4.0


In [6]:
# Export the Baseline assessment content (questions and related content)
baseline_assignment_questions.to_csv('raw_data/baseline_questions.csv', index=False)

#### Schools data
Dataset contains all schools from in the database regardless if they recieved the baseline assessment.

In [7]:
# Let's query the schools in south africa,
# making sure were remove schools with null school_uuid
master_schools_query = """
    select
        ms.school_uuid, ms.id as master_school_id, ms.display_name as school,
        co.full_name as country, ms.province_state, ms.district,ms.city_town,           
        ss.quintile, ss.urban_rural, ss.sector, ss.gis_longitude, ss.gis_latitude
    from master_schools ms
    left join countries co on co.id = ms.country
    left join sa_schools ss on ss.master_school_id = ms.id;
"""
master_schools= pd.read_sql(master_schools_query, conn)
master_schools.head()

,school_uuid,master_school_id,school,country,province_state,district,city_town,quintile,urban_rural,sector,gis_longitude,gis_latitude
0,979487a0-0c73-4f56-9fb8-0480393702cd,27686,Calling Academy (Stellenbosch),South Africa,Western Cape,Cape Winelands,Stellenbosch,5.0,Rural,Public,NaN,NaN
1,95dba0df-b656-4691-80c6-f33403d4b835,27688,Busy Bee Learning Academy (Johannesburg),South Africa,Gauteng,Johannesburg East,Johannesburg,NaN,Urban,Independent,NaN,NaN
2,204d5754-976c-4212-9ffc-4e6093d9d82c,27689,Lantern School of Excellence (Roodepoort),South Africa,Gauteng,Johannesburg West,Roodepoort,NaN,Urban,Independent,NaN,NaN
3,0dfbcbe0-5d42-45f5-b82a-d0b2ac05acf9,27690,EduExcellence (Noordhoek Campus) (Cape Town),South Africa,Western Cape,None,Cape Town,NaN,Urban,Independent,NaN,NaN
4,fc1fb02e-f622-4aa4-8264-00141839ba5c,13,Enhlanhleni Combined School (Underberg),South Africa,KwaZulu-Natal,Harry Gwala,Underberg,3.0,Rural,Public,29.6135,-29.7916


In [8]:
# Export the Baseline assessment content (questions and related content)
master_schools.to_csv('raw_data/master_schools.csv', index=False)

#### Baseline Assignment Recipients data
Dataset contains all schools that have had any of the baseline questions examined regardless if they are from Guateng Province or not.

In [9]:
# Let's query the assignment recipients (schools) of the basement assessment questions
# The baseline assessment is 22620 assignment
baseline_questions_recipients_query = """
    -- school that have ever recieved any baseline questions as part of any assignments
    select
        distinct cast(left(recipient_id, 36) as uuid) as school_uuid
    from assignment_recipient ar
    where assignment_id in  (
        -- assignments with content baseline questions as part of their content
        select distinct assignment_id
        from assignment_content ac1
        where exists (
            select template_id, section_id, assignment_id
            from assignment_content ac2
            where ac2.template_id = ac1.template_id
                and ac2.section_id = ac1.section_id
                and ac2.assignment_id = 22620));
"""
baseline_questions_recipients = pd.read_sql(baseline_questions_recipients_query, conn)
baseline_questions_recipients.head()

,school_uuid
0,002c567f-af10-4a61-bb2d-834cec705e73
1,002d436b-7b3c-4a5d-a269-0b3f63582b92
2,007f47e1-cb4e-4aef-83fc-16a8b81467ec
3,0095be15-1b5a-43d5-b9e8-705a57108850
4,00bf39bb-0e0e-4de2-9564-5edb79f869b4


In [10]:
# Export the Baseline assessment content (questions and related content)
baseline_questions_recipients.to_csv('raw_data/assignment_recipients.csv', index=False)

#### Baseline Activities data

In [11]:
# Let's query the activities related to the basement assessment
# The baseline assessment is 22620 assignment
baseline_activities_query = """
    select 
        aa.activity_uuid, aa.assignment_id , act.user_uuid,
        act.current_template_response_uuid, act.current_activity_template_id
    from activity_assignment aa
    left join activities act on act.uuid = aa.activity_uuid
    where assignment_id in (
        -- assignments with content baseline questions as part of their content
        select distinct assignment_id
        from assignment_content ac1
        where exists (
            select template_id, section_id, assignment_id
            from assignment_content ac2
            where ac2.template_id = ac1.template_id
                and ac2.section_id = ac1.section_id
                and ac2.assignment_id = 22620))
"""
baseline_activities = pd.read_sql(baseline_activities_query, conn)
baseline_activities.head()

DatabaseError: Execution failed on sql: 
    select 
        aa.activity_uuid, aa.assignment_id , act.user_uuid,
        act.current_template_response_uuid, act.current_activity_template_id
    from activity_assignment aa
    left join activities act on act.uuid = aa.activity_uuid
    where assignment_id in (
        -- assignments with content baseline questions as part of their content
        select distinct assignment_id
        from assignment_content ac1
        where exists (
            select template_id, section_id, assignment_id
            from assignment_content ac2
            where ac2.template_id = ac1.template_id
                and ac2.section_id = ac1.section_id
                and ac2.assignment_id = 22620))

SSL SYSCALL error: EOF detected

unable to rollback

In [ ]:
# Export the Baseline assessment content (questions and related content)
baseline_activities.to_csv('raw_data/baseline_activities.csv', index=False)

#### Students user data
Dataset contains information on student users

In [ ]:
# Let's query the user general profiles of users from Gauteng province in South Africa
user_query = """
    select
        uuid
    from users u
    where exists (
        select user_data_type, user_uuid
        from user_profile_general upg
        where u.uuid = upg.user_uuid
            and upg.user_data_type isnull)
    and u.role_id = 0;
    """
users = pd.read_sql(user_query, conn)
users.head()

In [ ]:
# Let's query the user general profiles of users from Gauteng province in South Africa
user_general_profiles_query = """
    select
        distinct user_uuid, master_school_id, grade,
        cast(grade_confirmed_at as date), cast(created_at as date)
    from user_profile_general upg
    where exists (
        select *
        from users u
        where u.uuid = upg.user_uuid
        and u.role_id = 0)
    and user_data_type is null
    order by user_uuid, created_at desc;
    """
user_general_profiles = pd.read_sql(user_general_profiles_query, conn)
user_general_profiles.head()

In [ ]:
# Export the Baseline assessment content (questions and related content)
users.to_csv('raw_data/siyavula_users.csv', index=False)
user_general_profiles.to_csv('raw_data/siyavula_user_details.csv', index=False)

#### Baseline responses data
Dataset coontains all responses from students (users) attributed to the questions attributed to the baseline assessment since 2018

In [ ]:
# Let's query the responses related to the basement activities
# The baseline assessment is 22620 assignment
baseline_responses_query = """
    with baseline_questions as (
        select *
        from assignment_content
        where assignment_id = 22620)
    select
        r.uuid, r.user_uuid, cast(r.created_at as date) as date, 
        bq.id as question_id, r.template_id, r.section_id, r.activity_uuid,
        aa.assignment_id, r.difficulty, r.attempted, r.correctness, r.durations
    from responses r
    inner join baseline_questions bq on (bq.template_id = r.template_id and bq.section_id = r.section_id)
    left join activity_assignment aa on aa.activity_uuid = r.activity_uuid
    where aa.assignment_id in (
        -- assignments with content baseline questions as part of their content
        select distinct assignment_id
        from assignment_content ac1
        where exists (
            select template_id, section_id, assignment_id
            from assignment_content ac2
            where ac2.template_id = ac1.template_id
                and ac2.section_id = ac1.section_id
                and ac2.assignment_id = 22620))
    and exists (
        -- where user roles are 0
        select u.uuid, u.role_id
        from users u
        where u.uuid = r.user_uuid
            and u.role_id = 0)
    and exists (
        -- where user profiles are null
        select upg.user_uuid, upg.user_data_type
        from user_profile_general upg
        where upg.user_uuid = r.user_uuid
            and upg.user_data_type isnull)
    and r.created_at >= '2019-01-01';
"""
baseline_responses = pd.read_sql(baseline_responses_query, conn)
baseline_responses.head()

# select
#     date, question_id, user_uuid, template_id, section_id, activity_uuid, assignment_id, difficulty,
#     sum(case when attempted is true then 1 else 0 end) as n_attempts,
#     sum(case when uuid is not null then 1 else 0 end) as n_responses, 
#     sum(case when correctness is true then 1 else 0 end) as n_correct_responses
# from baseline_content_responses
# group by (date, question_id, user_uuid, template_id, section_id, activity_uuid, assignment_id, difficulty);

In [ ]:
# Export the Baseline assessment content (questions and related content)
baseline_responses.to_csv('raw_data/responses.csv', index=False)